# Multi-Object Tracking Project
Each cell represents a method with different building blocks. You can run them seperately (like different `.py` files). 
> **Consideration:**
> 
> It is better to shutdown other kernels if you are a notebook user, and restart the kernel.
## Goal: 
There are circle-shaped markers on a single device. In a stereo setup, we are looking for the coordinations of these markers. Then, by grabing the coordinates with an approximately identican timing, calculate the position of that object in a 3D space.

#### #TODO:
- .py version with modularization and be capable of parsing parameters in terminal
- There is another options: choose specific trackers for every single object choosen.
- this is for oop code for multiobject tracking. check this for more development in the future codes.
https://github.com/opencv/opencv-python/issues/695


<br>
<br>

## Table of Contents

* [APIs with Manualy Selection in First Frame](#api_with_manualy_selection)
* [APIs with Automaticaly Selection in First Frame Hough Circle Transform](#api_with_automaticaly_detection_hough)
* * [APIs with Automaticaly Selection in First Frame Find ](#api_with_automaticaly_detection_hough) 

<a class="anchor" id="api_with_manualy_selection"></a>

---

### APIs with Manualy Selection in First Frame
#### **Running Process**:

First of all the firt frame of the first camera is appeared, then ROIs can be selected manualy(after each selection, press `Enter` or `Space`). Then, press `Esc` to pass this process on the second frame of the second camera. 
After pressing `Esc` againg, 2 windows are shown up with bounding boxes tracking selected objects.

>**NOTE:**
>
> It is important to select the objects in the same order between the first frames in first camera and second camera.


#### **Output format**: 
There are two lists named: `bboxes_centers1` and `bboxes_centers2` for center of objects in first camera and second camera respectively.

In each list, every single elements has this structure --> `(x_of_center, y_of_center, time_stamp, index_of_object)`

So, It is possible to distinguish the corresponding coordinates of every single object between the two lists(two cameras) by comparing `time_stamp` and `index_of_object`. 

In [29]:
import cv2
import time

# Function for choosing number of different types of tracker APIs
def ask_for_tracker():
    # TODO: Add GOTURN, MOSSE, CSRT
    print("Tracker APIs available: ")
    print("Enter 0 for BOOSTING: ")
    print("Enter 1 for MIL: ")
    print("Enter 2 for KCF: ")
    print("Enter 3 for TLD: ")
    print("Enter 4 for MEDIANFLOW: ")
    choice = input("Please select your tracker: ")
    
    return choice


# Function for setting different types of tracker APIs
def create_tracker(choice):
    if choice == '0':
        tracker = cv2.legacy.TrackerBoosting_create()
    if choice == '1':
        tracker = cv2.TrackerMIL_create()
    if choice == '2':
        tracker = cv2.legacy.TrackerKCF_create()
    if choice == '3':
        tracker = cv2.legacy.TrackerTLD_create()
    if choice == '4':
        # tracker = cv2.TrackerMedianFlow_create()
        tracker = cv2.legacy.TrackerMedianFlow_create()
    
    return tracker




# Ask for Tracker 
tracker_number = ask_for_tracker()
#tracker_name = str(tracker).split()[1]


# Create a MultiTracker object
multi_tracker1 = cv2.legacy.MultiTracker_create()
multi_tracker2 = cv2.legacy.MultiTracker_create()
#multi_tracker = []

# Read video
# First Camera
cap1 = cv2.VideoCapture()
cap1.open("/dev/v4l/by-path/pci-0000:00:14.0-usb-0:5:1.0-video-index0")
# cap1 = cv2.VideoCapture(cv2.CAP_V4L2 + 0)

# Read video
# Second Camera
cap2 = cv2.VideoCapture()
cap2.open("/dev/v4l/by-path/pci-0000:00:14.0-usb-0:6:1.0-video-index0")
# cap2 = cv2.VideoCapture(cv2.CAP_V4L2 + 2)


# Read first frame.
ret1, frame1 = cap1.read()

# Read first frame.
ret2, frame2 = cap2.read()

# Select multiple ROIs in the first frame for the first camera
# bboxes elements' structure --> (x, y, w, h)
bboxes1 = cv2.selectROIs("Select ROIs", frame1)
cv2.destroyWindow("Select ROIs")

# Select multiple ROIs in the first frame for the second camera
# bboxes elements' structure --> (x, y, w, h)
bboxes2 = cv2.selectROIs("Select ROIs", frame2)
cv2.destroyWindow("Select ROIs")


# Initialize trackers for selected ROIs(first camera)
# It is essential to create new tracker objects in each iteration.
for bbox in bboxes1:
    tracker = create_tracker(tracker_number)
    multi_tracker1.add(tracker, frame1, tuple(bbox))

# Initialize trackers for selected ROIs(second camera)
# It is essential to create new tracker objects in each iteration.
for bbox in bboxes2:
    tracker = create_tracker(tracker_number)
    multi_tracker2.add(tracker, frame2, tuple(bbox))

# Initialize a list for getting center of objects
bboxes_centers1 = []
bboxes_centers2 = []


start_time = time.time()

while True:
    # Read a new frame
    ret1, frame1 = cap1.read()
    ret2, frame2 = cap2.read()

    # Update the tracker with the current frame
    success1, bboxes1 = multi_tracker1.update(frame1)
    time_stamp1 = time.time() - start_time
    success2, bboxes2 = multi_tracker2.update(frame2)
    time_stamp2 = time.time() - start_time
    
    # first camera
    if success1:
        # Draw bounding boxes
        # Roi variable is a tuple of 4 floats, we need them as int
        for index in range(len(bboxes1)):
            (x1, y1, w1, h1) = [int(i) for i in bboxes1[index]]
            cv2.rectangle(frame1, (x1, y1), (x1 + w1, y1 + h1), (0, 255, 0), 2)
            bboxes_centers1.append((x1+(w1 / 2), y1+(h1 / 2), time_stamp1, index))
            
    else:
        # Tracking failure
        cv2.putText(frame1, "Failure to Detect Tracking!", (100,200), cv2.FONT_HERSHEY_SIMPLEX, 1,(0,0,255),3)

    
    # second camera    
    if success2:
        # Draw bounding boxes
        # Roi variable is a tuple of 4 floats, we need them as int
        for index in range(len(bboxes2)):
            (x2, y2, w2, h2) = [int(i) for i in bboxes2[index]]
            cv2.rectangle(frame2, (x2, y2), (x2 + w2, y2 + h2), (0, 255, 0), 2)
            bboxes_centers2.append((x2+(w2 / 2), y2+(h2 / 2), time_stamp2, index))
            
    else:
        # Tracking failure
        cv2.putText(frame2, "Failure to Detect Tracking!", (100,200), cv2.FONT_HERSHEY_SIMPLEX, 1,(0,0,255),3)


    
    # Display result
    tracker_name = str(tracker).split()[1]
    cv2.imshow(tracker_name + 'camera 1', frame1)
    cv2.imshow(tracker_name + 'camera 2', frame2)
    
    
    # Exit if ESC pressed
    if (cv2.waitKey(30) & 0xff) == 27: 
        break

cap1.release()
cap2.release() 
cv2.destroyAllWindows()

Tracker APIs available: 
Enter 0 for BOOSTING: 
Enter 1 for MIL: 
Enter 2 for KCF: 
Enter 3 for TLD: 
Enter 4 for MEDIANFLOW: 


Please select your tracker:  0


Finish the selection process by pressing ESC button!
Select a ROI and then press SPACE or ENTER button!
Cancel the selection process by pressing c button!
Select a ROI and then press SPACE or ENTER button!
Cancel the selection process by pressing c button!
Finish the selection process by pressing ESC button!
Select a ROI and then press SPACE or ENTER button!
Cancel the selection process by pressing c button!
Select a ROI and then press SPACE or ENTER button!
Cancel the selection process by pressing c button!


[ WARN:0@4419.084] global /croot/opencv-suite_1676452025216/work/modules/videoio/src/cap_gstreamer.cpp (2401) handleMessage OpenCV | GStreamer warning: Embedded video playback halted; module source reported: Could not read from resource.
[ WARN:0@4419.084] global /croot/opencv-suite_1676452025216/work/modules/videoio/src/cap_gstreamer.cpp (1356) open OpenCV | GStreamer warning: unable to start pipeline
[ WARN:0@4419.084] global /croot/opencv-suite_1676452025216/work/modules/videoio/src/cap_gstreamer.cpp (862) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created
[ WARN:0@4419.384] global /croot/opencv-suite_1676452025216/work/modules/videoio/src/cap_gstreamer.cpp (2401) handleMessage OpenCV | GStreamer warning: Embedded video playback halted; module source reported: Could not read from resource.
[ WARN:0@4419.385] global /croot/opencv-suite_1676452025216/work/modules/videoio/src/cap_gstreamer.cpp (1356) open OpenCV | GStreamer warning: unable to start 

<a class="anchor" id="api_with_automaticaly_detection_hough"></a>

---

### APIs with Automaticaly Selection in First Frame Hough Circle Transform


In [1]:
import cv2
import time

# Function for choosing number of different types of tracker APIs
def ask_for_tracker():
    # TODO: Add GOTURN, MOSSE, CSRT
    print("Tracker APIs available: ")
    print("Enter 0 for BOOSTING: ")
    print("Enter 1 for MIL: ")
    print("Enter 2 for KCF: ")
    print("Enter 3 for TLD: ")
    print("Enter 4 for MEDIANFLOW: ")
    choice = input("Please select your tracker: ")
    
    return choice


# Function for setting different types of tracker APIs
def create_tracker(choice):
    if choice == '0':
        tracker = cv2.legacy.TrackerBoosting_create()
    if choice == '1':
        tracker = cv2.TrackerMIL_create()
    if choice == '2':
        tracker = cv2.legacy.TrackerKCF_create()
    if choice == '3':
        tracker = cv2.legacy.TrackerTLD_create()
    if choice == '4':
        # tracker = cv2.TrackerMedianFlow_create()
        tracker = cv2.legacy.TrackerMedianFlow_create()
    
    return tracker




# Ask for Tracker 
tracker_number = ask_for_tracker()
#tracker_name = str(tracker).split()[1]


# Create a MultiTracker object
multi_tracker1 = cv2.legacy.MultiTracker_create()
multi_tracker2 = cv2.legacy.MultiTracker_create()
#multi_tracker = []

# Read video
# First Camera
cap1 = cv2.VideoCapture()
cap1.open("/dev/v4l/by-path/pci-0000:00:14.0-usb-0:5:1.0-video-index0")
# cap1 = cv2.VideoCapture(cv2.CAP_V4L2 + 0)

# Read video
# Second Camera
cap2 = cv2.VideoCapture()
cap2.open("/dev/v4l/by-path/pci-0000:00:14.0-usb-0:6:1.0-video-index0")
# cap2 = cv2.VideoCapture(cv2.CAP_V4L2 + 2)


# Read first frame.
ret1, frame1 = cap1.read()

# Read first frame.
ret2, frame2 = cap2.read()

# Select multiple ROIs in the first frame for the first camera
# bboxes elements' structure --> (x, y, w, h)
bboxes1 = cv2.selectROIs("Select ROIs", frame1)
cv2.destroyWindow("Select ROIs")

# Select multiple ROIs in the first frame for the second camera
# bboxes elements' structure --> (x, y, w, h)
bboxes2 = cv2.selectROIs("Select ROIs", frame2)
cv2.destroyWindow("Select ROIs")


# Initialize trackers for selected ROIs(first camera)
# It is essential to create new tracker objects in each iteration.
for bbox in bboxes1:
    tracker = create_tracker(tracker_number)
    multi_tracker1.add(tracker, frame1, tuple(bbox))

# Initialize trackers for selected ROIs(second camera)
# It is essential to create new tracker objects in each iteration.
for bbox in bboxes2:
    tracker = create_tracker(tracker_number)
    multi_tracker2.add(tracker, frame2, tuple(bbox))

# Initialize a list for getting center of objects
bboxes_centers1 = []
bboxes_centers2 = []


start_time = time.time()

while True:
    # Read a new frame
    ret1, frame1 = cap1.read()
    ret2, frame2 = cap2.read()

    # Update the tracker with the current frame
    success1, bboxes1 = multi_tracker1.update(frame1)
    time_stamp1 = time.time() - start_time
    success2, bboxes2 = multi_tracker2.update(frame2)
    time_stamp2 = time.time() - start_time
    
    # first camera
    if success1:
        # Draw bounding boxes
        # Roi variable is a tuple of 4 floats, we need them as int
        for index in range(len(bboxes1)):
            (x1, y1, w1, h1) = [int(i) for i in bboxes1[index]]
            cv2.rectangle(frame1, (x1, y1), (x1 + w1, y1 + h1), (0, 255, 0), 2)
            bboxes_centers1.append((x1+(w1 / 2), y1+(h1 / 2), time_stamp1, index))
            
    else:
        # Tracking failure
        cv2.putText(frame1, "Failure to Detect Tracking!", (100,200), cv2.FONT_HERSHEY_SIMPLEX, 1,(0,0,255),3)

    
    # second camera    
    if success2:
        # Draw bounding boxes
        # Roi variable is a tuple of 4 floats, we need them as int
        for index in range(len(bboxes2)):
            (x2, y2, w2, h2) = [int(i) for i in bboxes2[index]]
            cv2.rectangle(frame2, (x2, y2), (x2 + w2, y2 + h2), (0, 255, 0), 2)
            bboxes_centers2.append((x2+(w2 / 2), y2+(h2 / 2), time_stamp2, index))
            
    else:
        # Tracking failure
        cv2.putText(frame2, "Failure to Detect Tracking!", (100,200), cv2.FONT_HERSHEY_SIMPLEX, 1,(0,0,255),3)


    
    # Display result
    tracker_name = str(tracker).split()[1]
    cv2.imshow(tracker_name + 'camera 1', frame1)
    cv2.imshow(tracker_name + 'camera 2', frame2)
    
    # Exit if ESC pressed
    if (cv2.waitKey(30) & 0xff) == 27: 
        break

cap1.release()
cap2.release()
cv2.destroyAllWindows()

Tracker APIs available: 
Enter 0 for BOOSTING: 
Enter 1 for MIL: 
Enter 2 for KCF: 
Enter 3 for TLD: 
Enter 4 for MEDIANFLOW: 


Please select your tracker:  0


[ WARN:0@1.722] global /croot/opencv-suite_1676452025216/work/modules/videoio/src/cap_gstreamer.cpp (2401) handleMessage OpenCV | GStreamer warning: Embedded video playback halted; module source reported: Could not read from resource.
[ WARN:0@1.722] global /croot/opencv-suite_1676452025216/work/modules/videoio/src/cap_gstreamer.cpp (1356) open OpenCV | GStreamer warning: unable to start pipeline
[ WARN:0@1.722] global /croot/opencv-suite_1676452025216/work/modules/videoio/src/cap_gstreamer.cpp (862) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created

(python:12316): GStreamer-CRITICAL **: 04:15:47.910: gst_element_make_from_uri: assertion 'gst_uri_is_valid (uri)' failed
[ WARN:0@1.902] global /croot/opencv-suite_1676452025216/work/modules/videoio/src/cap_gstreamer.cpp (1127) open OpenCV | GStreamer warning: Error opening bin: no source element for URI "/dev/v4l/by-path/pci-0000:00:14.0-usb-0:6:1.0-video-index0"
[ WARN:0@1.902] global /croot/opencv-

error: OpenCV(4.6.0) /croot/opencv-suite_1676452025216/work/modules/highgui/src/window.cpp:967: error: (-215:Assertion failed) size.width>0 && size.height>0 in function 'imshow'
